In [1]:
# ! pip install qiskit-finance    # se estiver rodando no JupyterLab da IBM basta descomentar estas linhas
# ! pip install -U kaleido        # se estiver rodando no JupyterLab da IBM basta descomentar estas linhas
# ! pip install jinja2            # se estiver rodando no JupyterLab da IBM basta descomentar estas linhas
# ! pip install pylatexenc

# # se, no entanto, estiver rodando localmente é recomendado que seja instalado o python 3.10.11 e instaladas as bibliotecas no requirements.txt
# # !pip install -r requirements.txt

In [2]:
from qiskit.circuit.library import PauliTwoDesign, TwoLocal, EfficientSU2, RealAmplitudes
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, NFT
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA, SamplingVQE
from qiskit_aer.primitives import Sampler
from qiskit.quantum_info import Pauli

from IPython.display import clear_output

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import numpy as np
import pickle
import time

# Define Variaveis

In [3]:
ambiente_backend = 'local'
sampler = Sampler()

seed = 123
fator_risco = 0.5
records_tempo = []

quantidade_epocas_QAOA = 30
quantidade_epocas_VQE = 30

In [4]:
mostrar_figuras = True
salvar_figuras = False

width_padrao_display = 1400
height_padrao_display = 700

width_padrao_salvar = 1400
height_padrao_salvar = 700

width_padrao_display_junto = 3200
height_padrao_display_junto = 700

width_padrao_salvar_junto = 3200
height_padrao_salvar_junto = 700

tamanho_padrao_fonte = dict(size=35)
tamanho_padrao_margem = dict(l=90, r=90, t=90, b=90)

In [5]:
global lista_parametros_epoca_algoritmo_quantico
global lista_parametros_iteracao_otimizador
global otimizador_atual
global ansatz_atual
global mixer_atual

def guardar_parametros_iteracao_otimizador_QAOA(parametros):
    lista_parametros_iteracao_otimizador.append((otimizador_atual, mixer_atual, parametros))

def guardar_parametros_iteracao_otimizador_VQE(parametros):
    lista_parametros_iteracao_otimizador.append((otimizador_atual, ansatz_atual, parametros))

otimizadores_QAOA = [COBYLA(maxiter=500, callback=guardar_parametros_iteracao_otimizador_QAOA),
            L_BFGS_B(maxiter=500, callback=guardar_parametros_iteracao_otimizador_QAOA),
            SLSQP(maxiter=500, callback=guardar_parametros_iteracao_otimizador_QAOA),
            NFT(maxiter=500, callback=guardar_parametros_iteracao_otimizador_QAOA)]

otimizadores_VQE = [COBYLA(maxiter=500, callback=guardar_parametros_iteracao_otimizador_VQE), 
            L_BFGS_B(maxiter=500, callback=guardar_parametros_iteracao_otimizador_VQE), 
            SLSQP(maxiter=500, callback=guardar_parametros_iteracao_otimizador_VQE),
            NFT(maxiter=500, callback=guardar_parametros_iteracao_otimizador_VQE)]


# 4 Ativos

In [6]:
num_ativos = 4

In [7]:
df_vqe = pd.DataFrame()

with open('pickles/vqe_4_local.pickle', 'rb') as file:
    df_vqe = pickle.load(file)

In [8]:
# for otimizador in df_vqe['otimizador'].unique():
#     fig = go.Figure()

#     df_otimizador = df_vqe[df_vqe['otimizador'] == otimizador]

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='avaliacao_funcao_custo')

#         x_data = df_ansatz['avaliacao_funcao_custo']
#         y_data = df_ansatz['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Chamada de função de custo',
#         yaxis_title='Energia',
#         title='',
#         legend_title='Ansatzes',
#         width=width_padrao_display,
#         height=height_padrao_display,
#         font=tamanho_padrao_fonte,
#         margin=tamanho_padrao_margem)

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_funcao_custo_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [9]:
# df_vqe_real = df_vqe.copy()
# df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_vqe_real = df_vqe_real.sort_values(by='avaliacao_funcao_custo')

# fig = px.line(df_vqe_real, x='avaliacao_funcao_custo', y='valor_energia', color='ansatz',
#               facet_col='otimizador', facet_col_wrap=df_vqe_real['otimizador'].nunique(),
#               labels={'avaliacao_funcao_custo': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Chamada de função de custo',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_funcao_custo_juntos_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [10]:
# df_vqe_iteracao_otimizador = df_vqe.loc[df_vqe['bool_fim_iteracao_otimizador'] == True]

# for otimizador in df_vqe_iteracao_otimizador['otimizador'].unique():
#     fig = go.Figure()

#     df_otimizador = df_vqe_iteracao_otimizador[df_vqe_iteracao_otimizador['otimizador'] == otimizador]

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='iteracao_otimizador')

#         x_data = df_ansatz['iteracao_otimizador']
#         y_data = df_ansatz['valor_energia']

#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Iteração do otimizador',
#         yaxis_title='Energia',
#         title='',
#         legend_title='Ansatzes',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_otimizador_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [11]:
# df_vqe_real = df_vqe.copy()
# df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_vqe_real_iteracao_otimizador = df_vqe_real.loc[df_vqe_real['bool_fim_iteracao_otimizador'] == True]
# df_vqe_real_iteracao_otimizador = df_vqe_real_iteracao_otimizador.sort_values(by='iteracao_otimizador')

# fig = px.line(df_vqe_real_iteracao_otimizador, x='iteracao_otimizador', y='valor_energia', color='ansatz',
#               facet_col='otimizador', facet_col_wrap=df_vqe_real_iteracao_otimizador['otimizador'].nunique(),
#               labels={'iteracao_otimizador': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Iteração do otimizador',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_otimizador_juntos_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [12]:
# df_vqe_epoca = df_vqe.loc[df_vqe['nova_epoca'] == True]

# for otimizador in df_vqe_epoca['otimizador'].unique():
#     df_otimizador = df_vqe_epoca[df_vqe_epoca['otimizador'] == otimizador]

#     fig = go.Figure()

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='epoca')

#         x_data = df_ansatz['epoca']
#         y_data = df_ansatz['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Epoca',
#         yaxis_title='Energia',
#         title='',
#         legend_title='ansatzs',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_epoca_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [13]:
df_vqe_real = df_vqe.copy()
df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_vqe_real_epoca = df_vqe_real.loc[df_vqe_real['nova_epoca'] == True]
df_vqe_real_epoca = df_vqe_real_epoca.sort_values(by='epoca')
df_vqe_real_epoca = df_vqe_real_epoca.sort_values(by=['ansatz', 'otimizador'])


fig = px.line(df_vqe_real_epoca, x='epoca', y='valor_energia', color='ansatz',
              facet_col='otimizador', facet_col_wrap=df_vqe_real_epoca['otimizador'].nunique(),
              labels={'epoca': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=True, title_text='')

fig.add_annotation(
    text='Rodada de epocas (até 500 epocas por rodada)',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.17,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_epoca_junto_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [14]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['valor_otimo_encontrado'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='ansatz', y='valor_otimo_encontrado', color='ansatz', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Energia', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_valor_otimo_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [15]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['probabilidade_valor_otimo'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='probabilidade_valor_otimo', color='ansatz', facet_col='otimizador',
             title='', width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Probabilidade', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_probabilidade_valor_otimo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [16]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['tempo_elapsado_otimizador'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='ansatz', y='tempo_elapsado_otimizador', color='ansatz', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Tempo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_tempo_elapsado_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [17]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['quantidade_avaliacoes_total'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='quantidade_avaliacoes_total', color='ansatz', facet_col='otimizador',
             title='', width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Chamadas função custo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_chamadas_funcao_custo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [18]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['iteracao_otimizador'].max()

# fig = px.bar(df_agg, x='ansatz', y='iteracao_otimizador', color='ansatz', facet_col='otimizador',
#              title='', 
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas otimizador', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_chamadas_otimizador_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

### QAOA

In [19]:
df_qaoa = pd.DataFrame()

with open('pickles/qaoa_4_local.pickle', 'rb') as file:
    df_qaoa = pickle.load(file)

In [20]:
# for otimizador in df_qaoa['otimizador'].unique():
#     df_otimizador = df_qaoa[df_qaoa['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='avaliacao_funcao_custo')
        

#         x_data = df_mixer['avaliacao_funcao_custo']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Chamada de função de custo',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )


#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_funcao_custo_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [21]:
# df_qaoa_real = df_qaoa.copy()
# df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_qaoa_real = df_qaoa_real.sort_values(by='avaliacao_funcao_custo')

# fig = px.line(df_qaoa_real, x='avaliacao_funcao_custo', y='valor_energia', color='mixer',
#               facet_col='otimizador', facet_col_wrap=df_qaoa_real['otimizador'].nunique(),
#               labels={'avaliacao_funcao_custo': '', 'valor_energia': 'Energia'},
#               title='')


# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Chamada de função de custo',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_funcao_custo_junto_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [22]:
# df_qaoa_iteracao_otimizador = df_qaoa.loc[df_qaoa['bool_fim_iteracao_otimizador'] == True]

# for otimizador in df_qaoa_iteracao_otimizador['otimizador'].unique():
#     df_otimizador = df_qaoa_iteracao_otimizador[df_qaoa_iteracao_otimizador['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='iteracao_otimizador')

#         x_data = df_mixer['iteracao_otimizador']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Iteração do otimizador',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_otimizador_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [23]:
# df_qaoa_real = df_qaoa.copy()
# df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_qaoa_real_iteracao_otimizador = df_qaoa_real.loc[df_qaoa_real['bool_fim_iteracao_otimizador'] == True]
# df_qaoa_real_iteracao_otimizador = df_qaoa_real_iteracao_otimizador.sort_values(by='iteracao_otimizador')

# fig = px.line(df_qaoa_real_iteracao_otimizador, x='iteracao_otimizador', y='valor_energia', color='mixer',
#               facet_col='otimizador', facet_col_wrap=df_qaoa_real_iteracao_otimizador['otimizador'].nunique(),
#               labels={'iteracao_otimizador': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Iteração do otimizador',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_otimizador_junto_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [24]:
# df_qaoa_epoca = df_qaoa.loc[df_qaoa['nova_epoca'] == True]

# for otimizador in df_qaoa_epoca['otimizador'].unique():
#     df_otimizador = df_qaoa_epoca[df_qaoa_epoca['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='epoca')

#         x_data = df_mixer['epoca']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Epoca',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_epoca_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [25]:
df_qaoa_real = df_qaoa.copy()
df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_qaoa_real_epoca = df_qaoa_real.loc[df_qaoa_real['nova_epoca'] == True]
df_qaoa_real_epoca = df_qaoa_real_epoca.sort_values(by='epoca')

df_qaoa_real_epoca = df_qaoa_real_epoca.sort_values(by=['mixer', 'otimizador'])

fig = px.line(df_qaoa_real_epoca, x='epoca', y='valor_energia', color='mixer',
              facet_col='otimizador', facet_col_wrap=df_qaoa_real_epoca['otimizador'].nunique(),
              labels={'epoca': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=True, title_text='')

fig.add_annotation(
    text='Rodada de epocas (até 500 epocas por rodada)',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.17,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_epoca_junto_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [26]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['valor_otimo_encontrado'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='mixer', y='valor_otimo_encontrado', color='mixer', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Energia', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_valor_otimo_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [56]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['probabilidade_valor_otimo'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='probabilidade_valor_otimo', color='mixer', facet_col='otimizador',
             title='', width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Probabilidade', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.178,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_probabilidade_valor_otimo_qaoa.png" 
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [28]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['tempo_elapsado_otimizador'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='mixer', y='tempo_elapsado_otimizador', color='mixer', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Tempo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_tempo_elapsado_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [29]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['quantidade_avaliacoes_total'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='mixer', y='quantidade_avaliacoes_total', color='mixer', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas função custo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_chamadas_funcao_custo_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [30]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['iteracao_otimizador'].max()

# fig = px.bar(df_agg, x='mixer', y='iteracao_otimizador', color='mixer', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas otimizador', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_chamadas_otimizador_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

# 10 Ativos

In [31]:
lista_ativos = ['PETR4','VALE3','PRIO3','PETR3','GGBR4','JBSS3','VBBR3','BRFS3','CSAN3','KLBN4']
# lista_ativos = ['CSAN3', 'ITUB4', 'VALE3', 'KLBN4']

num_portfolios = 3417187 # 10 ativos
# num_portfolios = 29192 # 4 ativos

num_ativos = len(lista_ativos)
limite_ativos_comprar = 2
num_qubits = num_ativos

restricao_cardinalidade = False # se aplica apenas ao clássico, por default já temos no quantico

In [32]:
df_vqe = pd.DataFrame()

with open('pickles/vqe_10_local.pickle', 'rb') as file:
    df_vqe = pickle.load(file)


In [33]:
# for otimizador in df_vqe['otimizador'].unique():
#     fig = go.Figure()

#     df_otimizador = df_vqe[df_vqe['otimizador'] == otimizador]

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='avaliacao_funcao_custo')

#         x_data = df_ansatz['avaliacao_funcao_custo']
#         y_data = df_ansatz['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Chamada de função de custo',
#         yaxis_title='Energia',
#         title='',
#         legend_title='Ansatzes',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem)

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_funcao_custo_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [34]:
# df_vqe_real = df_vqe.copy()
# df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_vqe_real = df_vqe_real.sort_values(by='avaliacao_funcao_custo')

# fig = px.line(df_vqe_real, x='avaliacao_funcao_custo', y='valor_energia', color='ansatz',
#               facet_col='otimizador', facet_col_wrap=df_vqe_real['otimizador'].nunique(),
#               labels={'avaliacao_funcao_custo': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Chamada de função de custo',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_funcao_custo_juntos_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [35]:
# df_vqe_iteracao_otimizador = df_vqe.loc[df_vqe['bool_fim_iteracao_otimizador'] == True]

# for otimizador in df_vqe_iteracao_otimizador['otimizador'].unique():
#     fig = go.Figure()

#     df_otimizador = df_vqe_iteracao_otimizador[df_vqe_iteracao_otimizador['otimizador'] == otimizador]

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='iteracao_otimizador')

#         x_data = df_ansatz['iteracao_otimizador']
#         y_data = df_ansatz['valor_energia']

#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Iteração do otimizador',
#         yaxis_title='Energia',
#         title='',
#         legend_title='Ansatzes',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_otimizador_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [36]:
# df_vqe_real = df_vqe.copy()
# df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_vqe_real_iteracao_otimizador = df_vqe_real.loc[df_vqe_real['bool_fim_iteracao_otimizador'] == True]
# df_vqe_real_iteracao_otimizador = df_vqe_real_iteracao_otimizador.sort_values(by='iteracao_otimizador')

# fig = px.line(df_vqe_real_iteracao_otimizador, x='iteracao_otimizador', y='valor_energia', color='ansatz',
#               facet_col='otimizador', facet_col_wrap=df_vqe_real_iteracao_otimizador['otimizador'].nunique(),
#               labels={'iteracao_otimizador': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Iteração do otimizador',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_otimizador_juntos_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [37]:
# df_vqe_epoca = df_vqe.loc[df_vqe['nova_epoca'] == True]

# for otimizador in df_vqe_epoca['otimizador'].unique():
#     df_otimizador = df_vqe_epoca[df_vqe_epoca['otimizador'] == otimizador]

#     fig = go.Figure()

#     for ansatz in df_otimizador['ansatz'].unique():
#         df_ansatz = df_otimizador[df_otimizador['ansatz'] == ansatz]
#         df_ansatz = df_ansatz.sort_values(by='epoca')

#         x_data = df_ansatz['epoca']
#         y_data = df_ansatz['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=ansatz
#         ))

#     fig.update_layout(
#         xaxis_title='Epoca',
#         yaxis_title='Energia',
#         title='',
#         legend_title='ansatzs',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_epoca_vqe.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [38]:
df_vqe_real = df_vqe.copy()
df_vqe_real['valor_energia'] = df_vqe['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_vqe_real_epoca = df_vqe_real.loc[df_vqe_real['nova_epoca'] == True]
df_vqe_real_epoca = df_vqe_real_epoca.sort_values(by='epoca')

df_vqe_real_epoca = df_vqe_real_epoca.sort_values(by=['ansatz', 'otimizador'])

fig = px.line(df_vqe_real_epoca, x='epoca', y='valor_energia', color='ansatz',
              facet_col='otimizador', facet_col_wrap=df_vqe_real_epoca['otimizador'].nunique(),
              labels={'epoca': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='ansatzs', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=True, title_text='')

fig.add_annotation(
    text='Rodada de epocas (até 500 epocas por rodada)',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.17,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_energia_por_epoca_junto_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [39]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['valor_otimo_encontrado'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='ansatz', y='valor_otimo_encontrado', color='ansatz', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Energia', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_valor_otimo_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [40]:
df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['probabilidade_valor_otimo'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='ansatz', y='probabilidade_valor_otimo', color='ansatz', facet_col='otimizador',
             title='', width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Probabilidade', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Ansatz',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_probabilidade_valor_otimo_vqe.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [41]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['tempo_elapsado_otimizador'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='ansatz', y='tempo_elapsado_otimizador', color='ansatz', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Tempo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_tempo_elapsado_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [42]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['quantidade_avaliacoes_total'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='ansatz', y='quantidade_avaliacoes_total', color='ansatz', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas função custo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_chamadas_funcao_custo_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [43]:
# df_agg = df_vqe.groupby(['ansatz', 'otimizador'], as_index=False)['iteracao_otimizador'].max()

# fig = px.bar(df_agg, x='ansatz', y='iteracao_otimizador', color='ansatz', facet_col='otimizador',
#              title='', 
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas otimizador', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Ansatz',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_vqe/{num_ativos}_{ambiente_backend}_chamadas_otimizador_vqe.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

### QAOA

In [44]:
df_qaoa = pd.DataFrame()

with open('pickles/qaoa_10_local.pickle', 'rb') as file:
    df_qaoa = pickle.load(file)

In [45]:
# for otimizador in df_qaoa['otimizador'].unique():
#     df_otimizador = df_qaoa[df_qaoa['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='avaliacao_funcao_custo')
        

#         x_data = df_mixer['avaliacao_funcao_custo']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Chamada de função de custo',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )


#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_funcao_custo_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [46]:
# df_qaoa_real = df_qaoa.copy()
# df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_qaoa_real = df_qaoa_real.sort_values(by='avaliacao_funcao_custo')

# fig = px.line(df_qaoa_real, x='avaliacao_funcao_custo', y='valor_energia', color='mixer',
#               facet_col='otimizador', facet_col_wrap=df_qaoa_real['otimizador'].nunique(),
#               labels={'avaliacao_funcao_custo': '', 'valor_energia': 'Energia'},
#               title='')


# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Chamada de função de custo',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_funcao_custo_junto_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [47]:
# df_qaoa_iteracao_otimizador = df_qaoa.loc[df_qaoa['bool_fim_iteracao_otimizador'] == True]

# for otimizador in df_qaoa_iteracao_otimizador['otimizador'].unique():
#     df_otimizador = df_qaoa_iteracao_otimizador[df_qaoa_iteracao_otimizador['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='iteracao_otimizador')

#         x_data = df_mixer['iteracao_otimizador']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Iteração do otimizador',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_otimizador_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [48]:
# df_qaoa_real = df_qaoa.copy()
# df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
# df_qaoa_real_iteracao_otimizador = df_qaoa_real.loc[df_qaoa_real['bool_fim_iteracao_otimizador'] == True]
# df_qaoa_real_iteracao_otimizador = df_qaoa_real_iteracao_otimizador.sort_values(by='iteracao_otimizador')

# fig = px.line(df_qaoa_real_iteracao_otimizador, x='iteracao_otimizador', y='valor_energia', color='mixer',
#               facet_col='otimizador', facet_col_wrap=df_qaoa_real_iteracao_otimizador['otimizador'].nunique(),
#               labels={'iteracao_otimizador': '', 'valor_energia': 'Energia'},
#               title='')

# fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Iteração do otimizador',
#     xref='paper',
#     yref='paper',
    # x=0.5,
    # y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_otimizador_junto_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [49]:
# df_qaoa_epoca = df_qaoa.loc[df_qaoa['nova_epoca'] == True]

# for otimizador in df_qaoa_epoca['otimizador'].unique():
#     df_otimizador = df_qaoa_epoca[df_qaoa_epoca['otimizador'] == otimizador]

#     fig = go.Figure()

#     for mixer in df_otimizador['mixer'].unique():
#         df_mixer = df_otimizador[df_otimizador['mixer'] == mixer]
#         df_mixer = df_mixer.sort_values(by='epoca')

#         x_data = df_mixer['epoca']
#         y_data = df_mixer['valor_energia']

#         # Handling complex numbers by extracting real and imaginary parts
#         y_real = y_data.apply(lambda x: x.real if isinstance(x, complex) else x)

#         fig.add_trace(go.Scatter(
#             x=x_data,
#             y=y_real,
#             mode='lines',
#             name=mixer
#         ))

#     fig.update_layout(
#         xaxis_title='Epoca',
#         yaxis_title='Energia',
#         title='',
#         legend_title='mixers',
#         width=width_padrao_display,
#         height=height_padrao_display, 
#         font=tamanho_padrao_fonte, 
#         margin=tamanho_padrao_margem
#     )

#     if salvar_figuras:
#         nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_{otimizador}_energia_por_epoca_qaoa.png"
#         fig.write_image(nome_figura, width=width_padrao_salvar, height=height_padrao_salvar)

#     if mostrar_figuras:
#         fig.show()

In [50]:
df_qaoa_real = df_qaoa.copy()
df_qaoa_real['valor_energia'] = df_qaoa['valor_energia'].apply(lambda x: x.real if isinstance(x, complex) else x)
df_qaoa_real_epoca = df_qaoa_real.loc[df_qaoa_real['nova_epoca'] == True]
df_qaoa_real_epoca = df_qaoa_real_epoca.sort_values(by='epoca')

df_qaoa_real_epoca = df_qaoa_real_epoca.sort_values(by=['mixer', 'otimizador'])

fig = px.line(df_qaoa_real_epoca, x='epoca', y='valor_energia', color='mixer',
              facet_col='otimizador', facet_col_wrap=df_qaoa_real_epoca['otimizador'].nunique(),
              labels={'epoca': '', 'valor_energia': 'Energia'},
              title='')

fig.update_layout(width=width_padrao_display_junto, height=height_padrao_display_junto, legend_title='mixers', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=True, title_text='')

fig.add_annotation(
    text='Rodada de epocas (até 500 epocas por rodada)',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.17,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_energia_por_epoca_junto_qaoa.png"
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [51]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['valor_otimo_encontrado'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='mixer', y='valor_otimo_encontrado', color='mixer', facet_col='otimizador',
#              title='', width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Energia', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_valor_otimo_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [52]:
df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['probabilidade_valor_otimo'].mean() # pode ser média, max, min, pois todos os valores sao iguais

fig = px.bar(df_agg, x='mixer', y='probabilidade_valor_otimo', color='mixer', facet_col='otimizador',
             title='', width=width_padrao_display_junto, height=height_padrao_display_junto, 
             )

fig.update_layout(xaxis_title='', yaxis_title='Probabilidade', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
fig.update_xaxes(showticklabels=False, title_text='')

fig.add_annotation(
    text='Mixer',
    xref='paper',
    yref='paper',
    x=0.5,
    y=-0.13,
    showarrow=False,
    font=tamanho_padrao_fonte
)

if salvar_figuras:
    nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_probabilidade_valor_otimo_qaoa.png" 
    fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

if mostrar_figuras:
    fig.show()

In [53]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['tempo_elapsado_otimizador'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='mixer', y='tempo_elapsado_otimizador', color='mixer', facet_col='otimizador',
#              title='', width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Tempo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_tempo_elapsado_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [54]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['quantidade_avaliacoes_total'].mean() # pode ser média, max, min, pois todos os valores sao iguais

# fig = px.bar(df_agg, x='mixer', y='quantidade_avaliacoes_total', color='mixer', facet_col='otimizador',
#              title='', width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas função custo', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)
# fig.update_xaxes(showticklabels=False, title_text='')

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.13,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_chamadas_funcao_custo_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()

In [55]:
# df_agg = df_qaoa.groupby(['mixer', 'otimizador'], as_index=False)['iteracao_otimizador'].max()

# fig = px.bar(df_agg, x='mixer', y='iteracao_otimizador', color='mixer', facet_col='otimizador',
#              title='',
#              width=width_padrao_display_junto, height=height_padrao_display_junto, 
#              )

# fig.update_layout(xaxis_title='', yaxis_title='Chamadas otimizador', font=tamanho_padrao_fonte, margin=tamanho_padrao_margem)

# fig.add_annotation(
#     text='Mixer',
#     xref='paper',
#     yref='paper',
#     x=0.5,
#     y=-0.178,
#     showarrow=False,
#     font=tamanho_padrao_fonte
# )

# if salvar_figuras:
#     nome_figura = f"figuras_qaoa/{num_ativos}_{ambiente_backend}_chamadas_otimizador_qaoa.png"
#     fig.write_image(nome_figura, width=width_padrao_salvar_junto, height=height_padrao_salvar_junto)

# if mostrar_figuras:
#     fig.show()